In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from google.colab import drive
import tensorflow as tf
import numpy as np
import cv2
import os
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, Concatenate
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import LearningRateScheduler
import numpy as np
from time import time


drive.mount('/content/gdrive')

%cd gdrive/My Drive/Project ML

tf.logging.set_verbosity(tf.logging.INFO)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/Project ML


In [0]:
"""Load Training Data with Returning Features and Labels"""

def load_data(size):
  #load data

  images_path = os.listdir('progress 1/dummy_data/')
  
  label_map = {'free':[1,0,0],'thin':[0,1,0],'thick': [0,0,1]}
  feature = []
  label = []

  for file_name in images_path:
    
    # Read image file
    temp = cv2.imread('progress 1/dummy_data/' + file_name)
    
    # Resize image
    im_resize = cv2.resize(temp,(size,size))
    feature.append(im_resize)
    
    # Append labels.
    label.append(label_map[file_name.split('_')[0]])
   
  label = np.array(label).astype(float)
  return np.array(feature) / 255.0, label

In [0]:
def cnn_model_fn():
  model = Sequential()
  #Layer 1
  #Conv Layer 1
  model.add(Conv2D(filters = 6, 
                   kernel_size = 5, 
                   strides = 1, 
                   activation = 'relu', 
                   input_shape = (128,128,3)))
  #Pooling layer 1
  model.add(MaxPooling2D(pool_size = 2, strides = 2))
  #Layer 2
  #Conv Layer 2
  model.add(Conv2D(filters = 16, 
                   kernel_size = 5,
                   strides = 1,
                   activation = 'relu',
                   input_shape = (64,64,6)))
  #Pooling Layer 2
  model.add(MaxPooling2D(pool_size = 2, strides = 2))
  #Flatten
  model.add(Flatten())
  #Layer 3
  #Fully connected layer 1
  model.add(Dense(units = 120, activation = 'relu'))
  #Layer 4
  #Fully connected layer 2
  model.add(Dense(units = 84, activation = 'relu'))
  #Layer 5
  #Output Layer
  model.add(Dense(units = 3, activation = 'softmax'))

  model.summary()
  return model

In [0]:
def cnn_model_function(input_shape, filter_num, kernel_size):
  model = Sequential()
  #Layer 1
  #Conv Layer 1
  model.add(Conv2D(filters = filter_num[0], 
                   kernel_size = kernel_size[0], 
                   strides = 1, 
                   activation = 'relu', 
                   input_shape = (input_shape,input_shape,3)))
  #Pooling layer 1
  model.add(MaxPooling2D(pool_size = 2, strides = 2))
  #Layer 2
  #Conv Layer 2
  model.add(Conv2D(filters = filter_num[1], 
                   kernel_size = kernel_size[0],
                   strides = 1,
                   activation = 'relu',
                   input_shape = (input_shape/2,input_shape/2,6)))
  #Pooling Layer 2
  model.add(MaxPooling2D(pool_size = 2, strides = 2))
  #Flatten
  model.add(Flatten())
  return model

In [0]:
# Create a TensorBoard instance with the path to the logs directory
from tensorflow.keras.callbacks import TensorBoard
tensorboard = TensorBoard(log_dir='logs/{}'.format(time()))

In [0]:
# define the checkpoint
from tensorflow.keras.callbacks import ModelCheckpoint
filepath = "checkpoint/model.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

In [0]:
train_data, train_label = load_data(128)
train_data

In [0]:
train_data = {}
train_data, train_label = load_data(128)
#train_data, train_label = load_data(64)

model = cnn_model_fn() 
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(train_data ,train_label, batch_size = 64, epochs = 42, validation_data=(train_data,train_label))#, callbacks=[tensorboard, checkpoint])

In [0]:
#train_data = {}
train_data1, train_label = load_data(128)
train_data2, train_label = load_data(64)

cnn1 = cnn_model_function(128, [64, 256], [36, 2])
cnn2 = cnn_model_function(64, [32, 128], [12, 3])

concat_layer = Concatenate([cnn1, cnn2])


Using TensorFlow backend.


In [0]:
#testing
scores = model.evaluate(train_data, train_label, batch_size=1, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))
print(scores)

9/9 [==============================] - 0s 16ms/sample - loss: 1.9485e-05 - acc: 1.0000

Test result: 100.000 loss: 0.000
[1.948466899648338e-05, 1.0]
